In [173]:
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplplot
import dateutil.parser
import sys

In [174]:
verbose = True
if verbose :
    import pprint
    from IPython.core.display import display
    pprinter = pprint.PrettyPrinter(indent=4)

In [175]:
%matplotlib inline

In [176]:
sampleDataFileName = 'decoding-the-civil-war-classifications.csv'
liveDate = dateutil.parser.parse("2016-06-20T00:00:00.00Z")

In [177]:
class TextLine() :
    
    def __init__(self, x1, y1, x2, y2, text) :
        self.coords = {'start' : {'x' : x1, 'y' : y1}, 'end' : {'x' : x2, 'y' : y2}}
        self.text = text
        self.words = text.split()
        self.numWords = len(self.words)
        
    def __str__(self) :
        return str(self.text) + " @ ((" + str(self.coords['start']['x']) + ", " + str(self.coords['start']['y']) + "), (" + str(self.coords['end']['x']) + ", " + str(self.coords['end']['y']) + "))"
    
    def getStart(self) :
        return self.coords['start']
    
    def getEnd(self) :
        return self.coords['end']
    
    def getText(self) :
        return str(self.text)
    
    def getCoords(self) :
        return self.coords
    
    def getWords(self) :
        return self.words

In [178]:
class TelegramLines() :
    
    def __init__(self) :
        self.textLines = []
        
    def __str__(self) :
        return "\n".join([textLine.__str__() for textLine in self.textLines])
        
    def addLine(self, textLine) :
        self.textLines.append(textLine)
    
    def getLines(self) :
        return self.textLines
    
    def getNumLines(self) :
        return len(self.textLines)

In [179]:
telegrams = {}

with open(sampleDataFileName) as csvfile :
    parsedCsv = csv.DictReader(csvfile)
    nTelegramsParsed = 0
    for record in parsedCsv :
        done = False
        recordIsTelegram = True
        
        # check the date that the classification was made
        parsedMetadata = json.loads(record["metadata"])
        parsedDate = dateutil.parser.parse(parsedMetadata['started_at'])
        # skip "testing" data before the site went live
        
        if parsedDate < liveDate :
            continue
        
        # parse the annotations and the subject data
        parsedAnnotations = json.loads(record["annotations"])
        parsedSubjectData = json.loads(record["subject_data"])
        
        # initialize container for transcribed lines
        transcribedLines = TelegramLines()
        
        #loop over tasks in the annotation
        for task in parsedAnnotations :
            # Check if the current record is for a telegram (tasks may be stored out of order, so
            # some tasks may be processed before non-telegrams are caught - inefficient but unavoidable?)
            if task['task'] == "T1" and not task['value'].startswith("Telegram") :
                recordIsTelegram = False
                break
            
            # Process transcriptions of text lines
            if task['task'].startswith("T12") and len(task['value']) > 0 :
                # process the lines that were transcribed for this task
                for taskValueItem in task['value']:
                    transcribedLine = TextLine(taskValueItem['x1'], 
                                               taskValueItem['y1'], 
                                               taskValueItem['x2'], 
                                               taskValueItem['y2'],
                                               taskValueItem['details'][0]['value'])
                    transcribedLines.addLine(transcribedLine)
            
            # if the transcribed lines of a telegram have been processed then update the 
            # list of independent transcriptions for this subject
            if recordIsTelegram :
                nTelegramsParsed += 1
                if record['subject_ids'] in telegrams :
                    telegrams[record['subject_ids']].append(transcribedLines)
                else :
                    telegrams.update({record['subject_ids'] : [transcribedLines]})     

telegrams => {}


In [180]:
print(len(telegrams), nTelegramsParsed)

3662 50001


In [189]:
transcriptionLineStats = {}
transcriptionLineDetails = []
# loop over distinct subjects (currently individual telegram-type pages, codebook handling to be implemented)
for key, transcriptions in telegrams.items() :
    totalLines = 0
    maxLines = 0
    minLines = sys.maxsize
    # loop over individual transcriptions of the subject
    for transcription in transcriptions :
        # process overall transcription statistics for this subject
        numLines = transcription.getNumLines()
        totalLines += numLines 
        maxLines = numLines if numLines > maxLines else maxLines
        minLines = numLines if numLines < minLines else minLines
        # process the lines of the individual transcriptions of a subject
        for textLine in transcription.getLines() :
            # Add a dictionary describing the current line
            lineDescription = {'subjectKey' : key, 
                               'numLines' : numLines,
                               'x1' : textLine.getStart()['x'],
                               'y1' : textLine.getStart()['y'],
                               'x2' : textLine.getEnd()['x'],
                               'y2' : textLine.getEnd()['y'], 
                              'words' : textLine.getWords()}
            transcriptionLineDetails.append(lineDescription)
    transcriptionLineStats.update({key: {'minLines': minLines, 'maxLines':maxLines, 'meanLines':totalLines/float(len(transcriptions))}})

transcriptionLineDetailsFrame = pd.DataFrame(data=transcriptionLineDetails)
transcriptionLineDetailsIndex = pd.MultiIndex.from_arrays([transcriptionLineDetailsFrame['subjectKey'],
                                                           transcriptionLineDetailsFrame['y1'],
                                                           transcriptionLineDetailsFrame['y2'],
                                                           transcriptionLineDetailsFrame['x1'],
                                                           transcriptionLineDetailsFrame['x2']])
transcriptionLineDetailsFrame = transcriptionLineDetailsFrame.set_index(transcriptionLineDetailsIndex)
display(transcriptionLineDetailsFrame)
pprinter.pprint(transcriptionLineStats)

numLines  \
subjectKey y1          y2          x1          x2                      
2316884    169.878475  172.363913  577.456220  1298.233138        18   
           283.233820  282.115373  308.753225  1235.945599        18   
           338.037713  338.037713  310.990118  1278.446576        18   
           397.315393  397.315393  322.174586  1310.881532        18   
           453.237732  454.356179  323.293033  1266.143661        18   
           512.486798  512.486798  336.227288  1270.180963        18   
           569.509703  570.584079  324.128031  1270.652926        18   
           629.674739  627.525988  301.566143  1270.652926        18   
           747.856060  748.930436  739.911384  1276.024804        18   
           801.574842  799.426091  825.861432  908.588352         18   
           860.147187  858.580747  671.425977  933.021380         18   
           916.539018  919.671897  666.726658  926.755622         18   
           977.630168  979.196608  683.957493  972.965589         18   
           1037.154879 1038.721318 697.272229  846.867205         18   
           1029.322680 1029.322680 839.818227  969.049490         18   
           1089.630610 1091.980270 662.027339  942.420017         18   
           1151.504981 1151.504981 663.593779  907.175128         18   
           1207.113592 1209.463252 659.677680  914.224106         18   
           169.878475  172.363913  577.456220  1298.233138        18   
           283.233820  282.115373  308.753225  1235.945599        18   
           338.037713  338.037713  310.990118  1278.446576        18   
           397.315393  397.315393  322.174586  1310.881532        18   
           453.237732  454.356179  323.293033  1266.143661        18   
           512.486798  512.486798  336.227288  1270.180963        18   
           569.509703  570.584079  324.128031  1270.652926        18   
           629.674739  627.525988  301.566143  1270.652926        18   
           747.856060  748.930436  739.911384  1276.024804        18   
           801.574842  799.426091  825.861432  908.588352         18   
           860.147187  858.580747  671.425977  933.021380         18   
           916.539018  919.671897  666.726658  926.755622         18   
...                                                              ...   
1959631    791.049101  782.121336  255.957730  1260.331431        32   
           842.383753  832.340016  273.813263  1269.259197        32   
           885.906609  878.094814  276.045204  1248.055753        32   
           935.009319  937.241261  253.725789  1236.896045        32   
           989.691883  981.880088  267.117438  1259.215460        32   
           1033.214740 1042.142505 283.857000  1245.823811        32   
           1095.709108 1084.549401 264.885497  1274.839051        32   
           1137.000024 1137.000024 268.233409  1309.434145        32   
           1195.030499 1190.566617 255.957730  1282.650847        32   
           1240.785297 1231.857532 257.073701  1269.259197        32   
           1283.837792 1292.809027 1278.156599 264.407075         32   
           1338.786608 1337.665204 268.892693  1287.127833        32   
           1398.221042 1395.978233 262.164267  1260.214129        32   
           1801.926632 1806.412249 249.828819  1278.156599        32   
           1555.217660 1560.824682 244.221797  1331.984007        32   
           1699.878830 1707.728661 247.586010  1279.278003        32   
           1757.070455 1761.556073 255.435841  1279.278003        32   
           1446.441432 1447.562836 254.314436  1283.763620        32   
           1499.147439 1504.754461 240.857584  1260.214129        32   
           1601.576599 1606.437767 1266.780627 239.858862         32   
           1656.264743 1659.910620 250.796491  1283.794715        32   
2316049    214.207372  215.999903  690.175094  1312.175146         3   
           280.530994  275.153403  166.762947  1305.005117         3   
           319.966661  325.3

{   '1959274': {'maxLines': 27, 'meanLines': 18.7, 'minLines': 0},
    '1959277': {'maxLines': 34, 'meanLines': 20.125, 'minLines': 0},
    '1959279': {'maxLines': 29, 'meanLines': 28.2, 'minLines': 28},
    '1959281': {'maxLines': 12, 'meanLines': 4.333333333333333, 'minLines': 0},
    '1959286': {'maxLines': 41, 'meanLines': 35.0, 'minLines': 32},
    '1959290': {'maxLines': 31, 'meanLines': 18.555555555555557, 'minLines': 0},
    '1959291': {'maxLines': 26, 'meanLines': 16.333333333333332, 'minLines': 0},
    '1959292': {'maxLines': 25, 'meanLines': 25.0, 'minLines': 25},
    '1959295': {'maxLines': 31, 'meanLines': 12.5, 'minLines': 1},
    '1959298': {'maxLines': 17, 'meanLines': 8.5, 'minLines': 0},
    '1959302': {'maxLines': 29, 'meanLines': 23.4, 'minLines': 0},
    '1959303': {   'maxLines': 37,
                   'meanLines': 34.666666666666664,
                   'minLines': 31},
    '1959305': {   'maxLines': 18,
                   'meanLines': 16.666666666666668,
        